In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import requests, io, json
import random
import plotly.express as px

import geopandas as gpd
import shapely.geometry


In [2]:
from ev import *
from radius_map import *

In [3]:
# bring in EV Data

ev_raw = pd.read_csv('./app/data_files/ev_car_final.csv',index_col=0)

ev_raw,ev_range,ev_df = ev_data(ev_raw)

# US Zip Code and Lat / Long Dataset    
us_zip_lat_long_data = pd.read_csv('./app/data_files/us_zip_code_lat_long.csv'
                                   ,dtype={'ZIP': str,'LAT': float,'LNG': float})

In [23]:
us_zip_lat_long_data['ZIP'].unique()

array(['00601', '00602', '00603', ..., '99926', '99927', '99929'],
      dtype=object)

In [22]:
# # Get distinct Brands
brands = ev_df['Brand'].unique()
brands

array(['Aiways', 'Audi', 'BMW', 'Byton', 'Citroen', 'CUPRA', 'Dacia',
       'DS 3', 'Fiat', 'Ford', 'Honda', 'Hyundai', 'Hyundai ', 'Jaguar',
       'Kia', 'Kia ', 'Lexus', 'Lightyear', 'Lucid Air', 'Mazda',
       'Mercedes', 'Mercedes ', 'MG', 'MG ', 'Mini', 'Nissan', 'Opel',
       'Peugeot', 'Polestar', 'Porsche', 'Renault', 'SEAT', 'Skoda',
       'Skoda ', 'Smart', 'Sono', 'Tesla', 'Tesla ', 'Toyota',
       'Volkswagen', 'Volvo'], dtype=object)

In [5]:
# # ev_df.loc[(ev_df['Brand']== 'Ford')]
# brand_1 = 'Tesla'
# # brand_2 = 'Volkswagen'

In [6]:
# # # Get Distinct models from brands
# models = ev_df.index[(ev_df['Brand']== brand_1)].unique()
# models

In [7]:
## Car 1
# brand_1 = 'Ford'
car_1 = 'Mustang Mach-E SR AWD'

## Car 2
# brand_2 = 'Volkswagen'
car_2 = 'ID.4 GTX'

In [8]:
## Set Distiance
range_1 = get_range(ev_range,car_1)
range_2 = get_range(ev_range,car_2)

In [9]:
## Set Zip 
# zip_search = input("Zip: ")
zip = '32026'

In [10]:
# Plot Map
radius_map(zip,us_zip_lat_long_data,range_1,range_2)

In [24]:
c_table = get_comparison_table(ev_df,car_1,car_2)

In [73]:
a = c_table.to_dict()


In [75]:
for x,y in a.items():
        print(x,y)

Mustang Mach-E SR AWD {'Brand': 'Ford', 'Battery Size (kWh)': 68.0, 'Segment': 'D', 'Powertrian': 'AWD', 'BodyStyle': 'SUV', 'Seats': 5, '0 - 62 MPH (Sec)': '6.3', 'Top Speed (MPH)': '112', 'Range (Mi)': '200*', 'Efficiency (Wh/mi)': '330*', 'Euro': '€54,000'}
ID.4 GTX {'Brand': 'Volkswagen', 'Battery Size (kWh)': 77.0, 'Segment': 'C', 'Powertrian': 'AWD', 'BodyStyle': 'SUV', 'Seats': 5, '0 - 62 MPH (Sec)': '6.2', 'Top Speed (MPH)': '112', 'Range (Mi)': '245', 'Efficiency (Wh/mi)': '310', 'Euro': '€50,415'}


In [60]:
for a in a.items():
        print(x,y)

AttributeError: 'tuple' object has no attribute 'items'

In [59]:
for a in a.items():
    for b in a:
        print(b)

Mustang Mach-E SR AWD
{'Brand': 'Ford', 'Battery Size (kWh)': 68.0, 'Segment': 'D', 'Powertrian': 'AWD', 'BodyStyle': 'SUV', 'Seats': 5, '0 - 62 MPH (Sec)': '6.3', 'Top Speed (MPH)': '112', 'Range (Mi)': '200*', 'Efficiency (Wh/mi)': '330*', 'Euro': '€54,000'}
ID.4 GTX
{'Brand': 'Volkswagen', 'Battery Size (kWh)': 77.0, 'Segment': 'C', 'Powertrian': 'AWD', 'BodyStyle': 'SUV', 'Seats': 5, '0 - 62 MPH (Sec)': '6.2', 'Top Speed (MPH)': '112', 'Range (Mi)': '245', 'Efficiency (Wh/mi)': '310', 'Euro': '€50,415'}


In [50]:
for a in a.items():
    for b in a:
            for c in b:
                print(c)

M
u
s
t
a
n
g
 
M
a
c
h
-
E
 
S
R
 
A
W
D
Brand
Battery Size (kWh)
Segment
Powertrian
BodyStyle
Seats
0 - 62 MPH (Sec)
Top Speed (MPH)
Range (Mi)
Efficiency (Wh/mi)
Euro
I
D
.
4
 
G
T
X
Brand
Battery Size (kWh)
Segment
Powertrian
BodyStyle
Seats
0 - 62 MPH (Sec)
Top Speed (MPH)
Range (Mi)
Efficiency (Wh/mi)
Euro


In [40]:
for a in a:
    print(a)

Mustang Mach-E SR AWD
ID.4 GTX


In [11]:
# # formatted_table(comparison_table)
# display(get_comparison_table(ev_df,car_1,car_2))
# print("*** shaded values are etimates ***")

In [12]:
# get_comparison_table(ev_df,car_1,car_2)

In [13]:
# a = get_data_table(car_1,car_2)

In [14]:
# (a[car_1][1]-a[car_2][1])

In [15]:
# diff = []

# for i in range(0,11):
    
#     try:
#         b = (int(a[car_1][i])) - (int(a[car_2][i]))
#     except:
#         try:
#             b = round((float(a[car_1][i].replace("*",""))) - (float(a[car_2][i].replace("*",""))),2)
#         except:
#             b = "NA"
#     diff.append(b)

In [16]:
# diff

In [17]:
# (float(a[car_1][1])) - (float(a[car_2][1]))

In [18]:
# a[car_1]

In [19]:
# (float(a[car_1][1].replace("*",""))) - (float(a[car_2][1].replace("*","")))

In [20]:
# (int(a[car_1][2])) - (int(a[car_2][2]))
# (float(a[car_1][2].replace("*",""))) - (float(a[car_2][2].replace("*","")))